In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
np.set_printoptions(threshold=np.inf, suppress=True)
import matplotlib.pyplot as plt

Loading data

Dataset obsahuje 60 continues features a jeden predicate (R/M), ktorý vraví o tom, či sa jedná o kameň alebo mínu. 

In [2]:
sonar_data = pd.read_csv('dataset/sonar.all-data', header=None)
print(sonar_data.head())
print(sonar_data.shape)
print(sonar_data[60].value_counts())

       0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      51      52      53      54      55      56      57  \
0  0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084   
1  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
2  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
3  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
4  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   

       58      59  60  
0  0.0090  0.0032   R  
1  0.0052  0.0044   R  
2  0.0095  0.0078   

In [3]:
sonar_data.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


Vykonanie encodingu, kde mínu zakódujeme ako 1 a kameň ako 0

In [4]:
# replacnutie R za 0 a M za 1
# R - Rock M - Mina
sonar_data[60] = sonar_data[60].replace(['R', 'M'], [0, 1])
sonar_data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


Data normalization

Data, ktoré používame normalizujeme pomocou standard scaleru

In [5]:
normalized_dfMinMax = sonar_data.copy()
for x in range(60):
    normalized_dfMinMax[x] = MinMaxScaler().fit_transform(np.array(normalized_dfMinMax[x]).reshape(-1,1))

normalized_dfMinMax

normalized_df = sonar_data.copy()
for x in range(60):
    normalized_df[x] = StandardScaler().fit_transform(np.array(normalized_df[x]).reshape(-1,1))

normalized_df


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,-0.399551,-0.040648,-0.026926,-0.715105,0.364456,-0.101253,0.521638,0.297843,1.125272,0.021186,...,-1.115432,-0.597604,0.680897,-0.295646,1.481635,1.763784,0.069870,0.171678,-0.658947,0
1,0.703538,0.421630,1.055618,0.323330,0.777676,2.607217,1.522625,2.510982,1.318325,0.588706,...,-0.522349,-0.256857,-0.843151,0.015503,1.901046,1.070732,-0.472406,-0.444554,-0.419852,0
2,-0.129229,0.601067,1.723404,1.172176,0.400545,2.093337,1.968770,2.852370,3.232767,3.066105,...,1.017585,0.836373,-0.197833,1.231812,2.827246,4.120162,1.309360,0.252761,0.257582,0
3,-0.835555,-0.648910,0.481740,-0.719414,-0.987079,-1.149364,-0.193816,-0.084747,-1.000852,-0.610469,...,-0.137365,-1.009341,0.557326,-0.111785,-0.161060,-0.488635,-0.549875,-0.639154,1.034640,0
4,2.050790,0.856537,0.111327,-0.312227,-0.292365,-0.672796,-0.013735,1.317299,1.510531,1.772220,...,-1.073812,-0.753780,-0.060532,0.241793,-1.174638,-0.107456,-0.487900,0.447361,0.576375,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,-0.456232,-0.116681,-0.705146,-0.779738,-0.647842,0.990954,1.314965,0.407323,0.463980,0.448504,...,-0.189390,-0.129077,1.230104,-0.847228,0.328253,-0.228741,0.550172,1.841992,1.831621,1
204,0.136733,-0.861801,-0.366036,0.054026,0.014392,-0.148740,-0.369029,-0.388465,-0.635067,0.053253,...,-0.761663,-0.200066,0.351373,-0.422934,-0.335815,-0.765856,-0.735798,-0.282388,0.038412,1
205,1.004381,0.160078,-0.673843,-0.531979,-0.723629,0.212502,0.064137,-0.200113,-0.442014,0.332912,...,0.268428,-1.108725,-0.801960,-0.437077,0.118548,1.070732,0.906526,-0.039138,-0.678871,1
206,0.049533,-0.095392,0.134804,0.148821,-1.055648,0.522865,0.401585,-0.264859,0.139685,0.202404,...,-0.501539,-0.867363,0.227802,-0.804798,-0.825128,-0.765856,-0.007598,-0.704020,-0.340154,1


In [6]:
normalized_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
count,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,...,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,208.000000
mean,1.708035e-17,6.832142e-17,-1.195625e-16,1.622634e-16,-1.793437e-16,2.049643e-16,1.024821e-16,3.416071e-17,-3.757678e-16,3.416071e-17,...,1.024821e-16,3.416071e-17,-1.451830e-16,2.775558e-17,-2.391250e-16,3.416071e-17,-1.110223e-16,1.345078e-16,7.686159e-17,0.533654
std,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,...,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,0.500070
min,-1.206158e+00,-1.150725e+00,-1.104253e+00,-1.036115e+00,-1.236093e+00,-1.600493e+00,-1.921613e+00,-1.522110e+00,-1.443689e+00,-1.468833e+00,...,-1.313126e+00,-1.449472e+00,-1.364897e+00,-1.229092e+00,-1.366868e+00,-1.302971e+00,-1.185113e+00,-1.271603e+00,-1.176985e+00,0.000000
25%,-6.894939e-01,-6.686781e-01,-6.490624e-01,-6.359298e-01,-6.703975e-01,-6.367565e-01,-6.626732e-01,-6.400918e-01,-6.856590e-01,-7.232644e-01,...,-6.394049e-01,-7.999231e-01,-7.642025e-01,-7.270112e-01,-6.678488e-01,-7.138771e-01,-6.738235e-01,-6.918580e-01,-6.788714e-01,0.000000
50%,-2.774703e-01,-2.322506e-01,-2.486515e-01,-2.120457e-01,-2.292089e-01,-2.106432e-01,-2.400524e-01,-2.672134e-01,-2.180558e-01,-1.928459e-01,...,-2.102002e-01,-1.645716e-01,-2.252935e-01,-2.532164e-01,-2.396997e-01,-3.240352e-01,-3.329639e-01,-2.499546e-01,-2.405314e-01,1.000000
75%,2.784345e-01,2.893335e-01,3.682681e-01,2.285353e-01,4.524231e-01,5.012417e-01,5.232608e-01,4.096773e-01,4.692723e-01,4.507410e-01,...,3.438640e-01,5.950106e-01,4.886751e-01,3.973675e-01,4.112618e-01,4.513169e-01,3.719959e-01,3.865486e-01,4.020352e-01,1.000000
max,4.706053e+00,5.944643e+00,6.836142e+00,8.025419e+00,5.878863e+00,4.710224e+00,4.074573e+00,3.816498e+00,4.274237e+00,3.746234e+00,...,5.980752e+00,4.016680e+00,3.330819e+00,5.008027e+00,5.448568e+00,4.795888e+00,5.585599e+00,4.615037e+00,7.450343e+00,1.000000


Train test val split 80/10/10

Dáta delíme takto kvôli menšiemu počtu dostupných vzoriek

In [7]:
X = normalized_df.drop(columns=60, axis=1)
y = normalized_df[60]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [9]:
print(X_train.shape)
print(y_train.value_counts())
print('******')
print(X_test.shape)
print(y_test.value_counts())
print('******')
print(X_val.shape)
print(y_val.value_counts())

(166, 60)
1    91
0    75
Name: 60, dtype: int64
******
(21, 60)
1    11
0    10
Name: 60, dtype: int64
******
(21, 60)
0    12
1     9
Name: 60, dtype: int64


Tensorflow

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
import wandb
from wandb.keras import WandbCallback

Vytvorenie modelu

V rámci tensorflow časti vytvárame 5 rôznych modelov

Všetky majú rovnakú architektúru, len sa líšia v metódach použitých pri riešení problému overfittingu. Máme vstupnú vrstvu so 16 neuronmi, dve skryté vrstvy s 16 neurónmi a nakoniec výstupnú vrstvu s jedným neurónom.

Pri všetkých vrstvách používame aktivačnú funkciu ReLu až na poslednú, kde používame sigmoid.

Model 1 nepoužíva žiadnu metódu prevencie overfittingu

Model 2 používa metódu dropout, kde sme ako šancu vypadnuta neurónu dali 50%, dropout sa vykonáva pri prvej a druhej skrytej vrstve.

Model 3 používa early stopping, riadi sa podľa val_loss a parameter patience sme nastavili na 10 epoch.

Model 4 používa regularizáciu, ako hodnotu parametrov l1 a l2 sme zvolili hodnotu 0,005. Skúsili sme použiť aj vyššie hodnoty ale to viedlo k oveľa horším výsledkom, kde napríklad acc padlo približne na 55%.

Model 5 sme skúsili otestovať ako sa bude NN správať ak použijeme naraz dropout a early stopping, parametre sme použili rovnaké ako pri predchádzajúcich modeloch.





In [11]:
#nothing
tf_model1 = Sequential()
tf_model1.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model1.add(Dense(16, activation=tf.keras.activations.relu))
tf_model1.add(Dense(16, activation=tf.keras.activations.relu))
tf_model1.add(Dense(1, activation=tf.keras.activations.sigmoid))
#dropout
tf_model2 = Sequential()
tf_model2.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model2.add(Dropout(0.5))
tf_model2.add(Dense(16, activation=tf.keras.activations.relu))
tf_model2.add(Dropout(0.5))
tf_model2.add(Dense(16, activation=tf.keras.activations.relu))
tf_model2.add(Dense(1, activation=tf.keras.activations.sigmoid))
#early stopping
tf_model3 = Sequential()
tf_model3.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model3.add(Dense(16, activation=tf.keras.activations.relu))
tf_model3.add(Dense(16, activation=tf.keras.activations.relu))
tf_model3.add(Dense(1, activation=tf.keras.activations.sigmoid))
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#regularization
tf_model4 = Sequential()
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns), kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(1, activation=tf.keras.activations.sigmoid))
#dropout + earlystop
tf_model5 = Sequential()
tf_model5.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model5.add(Dropout(0.5))
tf_model5.add(Dense(16, activation=tf.keras.activations.relu))
tf_model5.add(Dropout(0.5))
tf_model5.add(Dense(16, activation=tf.keras.activations.relu))
tf_model5.add(Dense(1, activation=tf.keras.activations.sigmoid))



Ako loss funkciu sme zvolili binary crossentropy a ako optimizer Adam.

Na sledovanie NN sme vybrali 4 metriky Acc, MSE, precision a recall

In [12]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
tf_model1.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model2.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model3.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model4.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model5.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])

Batch size sme zvolili na 16.

Počet epoch trénovania sme zvolili 100, pri tomto zadaní by mal postačovať aj menší počet epoch ale kvôli krátkemu času behu sme nechali 100 epoch.

In [13]:
def fitModel(model, name, callbackArg):
    
    model.fit(
    X_train,
    y_train,
    batch_size=16,
    epochs=100,
    validation_data=(X_val,y_val),
    callbacks=callbackArg
    )

    model.save('./'+ name)
    

In [14]:

w = wandb.init(project='zadanie1',reinit=True)
w.config.epochs = 100
w.config.batch_size = 16



fitModel(tf_model1, 'model1', [WandbCallback()])

wandb: Currently logged in as: cajnyoby. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/100
 1/11 [=>............................] - ETA: 23s - loss: 0.6886 - accuracy: 0.5000 - mse: 0.2488 - precision: 0.5000 - recall: 0.8750WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0016s vs `on_train_batch_end` time: 0.0028s). Check your callbacks.


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best)... Done. 0.0s


11/11 [==============================] - 2s 209ms/step - loss: 0.2319 - accuracy: 0.9337 - mse: 0.0636 - precision: 0.9255 - recall: 0.9560 - val_loss: 0.4518 - val_accuracy: 0.7619 - val_mse: 0.1379 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 16/100
11/11 [==============================] - 0s 14ms/step - loss: 0.2045 - accuracy: 0.9458 - mse: 0.0542 - precision: 0.9457 - recall: 0.9560 - val_loss: 0.4547 - val_accuracy: 0.7619 - val_mse: 0.1357 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 17/100
11/11 [==============================] - 0s 11ms/step - loss: 0.1833 - accuracy: 0.9639 - mse: 0.0475 - precision: 0.9670 - recall: 0.9670 - val_loss: 0.4609 - val_accuracy: 0.7619 - val_mse: 0.1349 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 18/100
11/11 [==============================] - 0s 9ms/step - loss: 0.1643 - accuracy: 0.9699 - mse: 0.0415 - precision: 0.9778 - recall: 0.9670 - val_loss: 0.4650 - val_accuracy: 0.8095 - val_mse: 0.1352 - val_precision: 0.6923 - val_

INFO:tensorflow:Assets written to: ./model1\assets


INFO:tensorflow:Assets written to: ./model1\assets


In [15]:
fitModel(tf_model2, 'model2', [WandbCallback()])

Epoch 1/100
 1/11 [=>............................] - ETA: 22s - loss: 0.8070 - accuracy: 0.5625 - mse: 0.2808 - precision: 0.7083 - recall: 0.8947WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0020s vs `on_train_batch_end` time: 0.0047s). Check your callbacks.


11/11 [==============================] - 3s 62ms/step - loss: 0.7230 - accuracy: 0.5602 - mse: 0.2580 - precision: 0.5926 - recall: 0.8000 - val_loss: 0.6523 - val_accuracy: 0.7143 - val_mse: 0.2299 - val_precision: 0.6000 - val_recall: 1.0000
Epoch 2/100
11/11 [==============================] - 0s 9ms/step - loss: 0.7427 - accuracy: 0.5301 - mse: 0.2660 - precision: 0.5607 - recall: 0.6593 - val_loss: 0.6391 - val_accuracy: 0.7619 - val_mse: 0.2233 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 3/100
11/11 [==============================] - 0s 8ms/step - loss: 0.7078 - accuracy: 0.5843 - mse: 0.2528 - precision: 0.5982 - recall: 0.7363 - val_loss: 0.6261 - val_accuracy: 0.7619 - val_mse: 0.2169 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 4/100
11/11 [==============================] - 0s 7ms/step - loss: 0.7146 - accuracy: 0.5361 - mse: 0.2582 - precision: 0.5700 - recall: 0.6264 - val_loss: 0.6156 - val_accuracy: 0.7619 - val_mse: 0.2118 - val_precision: 0.6667 - val_recall

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best)... Done. 0.0s


11/11 [==============================] - 2s 244ms/step - loss: 0.4755 - accuracy: 0.7530 - mse: 0.1597 - precision: 0.7451 - recall: 0.8352 - val_loss: 0.4500 - val_accuracy: 0.7143 - val_mse: 0.1438 - val_precision: 0.6364 - val_recall: 0.7778
Epoch 29/100
 9/11 [=======================>......] - ETA: 0s - loss: 0.4905 - accuracy: 0.7292 - mse: 0.1659 - precision: 0.7317 - recall: 0.7792

...

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155752-zy14ggwx\files\model-best)... Done. 0.0s


11/11 [==============================] - 2s 242ms/step - loss: 0.3456 - accuracy: 0.8313 - mse: 0.1111 - precision: 0.8316 - recall: 0.8681 - val_loss: 0.4089 - val_accuracy: 0.8095 - val_mse: 0.1281 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 59/100
11/11 [==============================] - 0s 11ms/step - loss: 0.3775 - accuracy: 0.8373 - mse: 0.1230 - precision: 0.8556 - recall: 0.8462 - val_loss: 0.4193 - val_accuracy: 0.8095 - val_mse: 0.1302 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 60/100
11/11 [==============================] - 0s 12ms/step - loss: 0.3286 - accuracy: 0.8494 - mse: 0.1082 - precision: 0.8587 - recall: 0.8681 - val_loss: 0.4271 - val_accuracy: 0.8095 - val_mse: 0.1315 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 61/100
11/11 [==============================] - 0s 11ms/step - loss: 0.3179 - accuracy: 0.8494 - mse: 0.1019 - precision: 0.8438 - recall: 0.8901 - val_loss: 0.4315 - val_accuracy: 0.8095 - val_mse: 0.1320 - val_precision: 0.7778 - val

INFO:tensorflow:Assets written to: ./model2\assets


INFO:tensorflow:Assets written to: ./model2\assets


In [16]:
w.finish()
w = wandb.init(project='zadanie1',reinit=True)

accuracy,▃▅▇▇████████████████▁▂▂▂▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▆
epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇█▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇██
loss,▇▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▇▇▇▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃
mse,▇▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▇▇▇▆▅▄▅▄▅▄▄▃▃▄▃▃▃▃▃
precision,▂▅▇▇████████████████▁▂▂▂▃▄▅▅▅▄▆▅▆▆▆▆▇▇▇▆
recall,▄▆▇▇████████████████▁▁▁▃▄▅▅▅▆▆▄▅▄▆▅▆▆▇▆▆
val_accuracy,▁▃▃▃▄▄██████████████▃▄▃▁▁▁▃▃▃▃▃▄▄▄▄▆▆▄▆▆
val_loss,█▆▂▂▃▃▂▃▃▃▄▄▅▅▆▆▆▇▇▇▇▆▅▄▃▂▂▂▂▂▁▁▂▁▂▂▃▃▃▄
val_mse,█▆▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁█▆▆▅▄▄▄▄▄▄▃▃▃▃▃▃▃▄▃▃
val_precision,▁▂▂▂▃▃██████████████▂▅▂▁▁▁▃▃▃▅▅▆▆▆▆▇▇▆▇▇
val_recall,▃▆▆▆████████████████▆▆▆▃▃▃▃▃▃▁▁▃▃▃▃▆▆▃▆▆


In [17]:
fitModel(tf_model3, 'model3', [earlyStop,WandbCallback()])

Epoch 1/100
 1/11 [=>............................] - ETA: 21s - loss: 0.6901 - accuracy: 0.4375 - mse: 0.2528 - precision: 0.6842 - recall: 0.6842WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0026s vs `on_train_batch_end` time: 0.0031s). Check your callbacks.


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155944-fxm4iz2n\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155944-fxm4iz2n\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155944-fxm4iz2n\files\model-best)... Done. 0.0s


11/11 [==============================] - 5s 255ms/step - loss: 0.6845 - accuracy: 0.5241 - mse: 0.2483 - precision: 0.5789 - recall: 0.6600 - val_loss: 0.7438 - val_accuracy: 0.4286 - val_mse: 0.2730 - val_precision: 0.3333 - val_recall: 0.3333
Epoch 2/100
 1/11 [=>............................] - ETA: 0s - loss: 0.6479 - accuracy: 0.5000 - mse: 0.2357 - precision: 0.5455 - recall: 0.6667

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155944-fxm4iz2n\files\model-best\assets


...

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155944-fxm4iz2n\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155944-fxm4iz2n\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_155944-fxm4iz2n\files\model-best)... Done. 0.0s


11/11 [==============================] - 2s 216ms/step - loss: 0.2214 - accuracy: 0.9398 - mse: 0.0593 - precision: 0.9880 - recall: 0.9011 - val_loss: 0.6145 - val_accuracy: 0.7143 - val_mse: 0.2018 - val_precision: 0.6364 - val_recall: 0.7778
Epoch 18/100
11/11 [==============================] - 0s 11ms/step - loss: 0.2017 - accuracy: 0.9578 - mse: 0.0526 - precision: 0.9884 - recall: 0.9341 - val_loss: 0.6321 - val_accuracy: 0.7143 - val_mse: 0.2046 - val_precision: 0.6364 - val_recall: 0.7778
Epoch 19/100
11/11 [==============================] - 0s 11ms/step - loss: 0.1793 - accuracy: 0.9639 - mse: 0.0452 - precision: 1.0000 - recall: 0.9341 - val_loss: 0.6616 - val_accuracy: 0.7143 - val_mse: 0.2076 - val_precision: 0.6364 - val_recall: 0.7778
Epoch 20/100
11/11 [==============================] - 0s 11ms/step - loss: 0.1628 - accuracy: 0.9699 - mse: 0.0403 - precision: 0.9886 - recall: 0.9560 - val_loss: 0.6977 - val_accuracy: 0.7143 - val_mse: 0.2101 - val_precision: 0.6364 - val

INFO:tensorflow:Assets written to: ./model3\assets


INFO:tensorflow:Assets written to: ./model3\assets


In [18]:
w.finish()
w = wandb.init(project='zadanie1',reinit=True)

accuracy,▁▂▂▃▃▃▄▄▄▅▅▆▇▇▇▇▇██████████
epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
loss,██▇▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
mse,██▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
precision,▁▁▂▃▃▃▄▅▅▆▇▇▇██████████████
recall,▂▁▁▁▂▂▁▁▂▃▃▄▅▆▆▆▆▇▇████████
val_accuracy,▁▂▂▂▂▂▂▃▆▆▆▆███████████████
val_loss,▄▄▄▄▄▄▃▃▃▃▂▂▁▁▁▁▁▁▂▃▄▄▅▆▇▇█
val_mse,█▇▇▇▆▆▆▅▅▅▄▃▂▂▁▁▁▁▂▂▂▂▂▃▃▃▃
val_precision,▁▁▁▁▁▁▁▃▆▆▆▆███████████████
val_recall,▂▁▁▁▁▁▁▂▅▅▅▅███████████████


In [19]:
fitModel(tf_model4, 'model4', [WandbCallback()])

Epoch 1/100
 1/11 [=>............................] - ETA: 22s - loss: 2.2229 - accuracy: 0.5000 - mse: 0.2508 - precision: 0.5714 - recall: 0.5000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0008s vs `on_train_batch_end` time: 0.0013s). Check your callbacks.


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160036-qgxjbijr\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160036-qgxjbijr\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160036-qgxjbijr\files\model-best)... Done. 0.0s


11/11 [==============================] - 4s 178ms/step - loss: 2.2310 - accuracy: 0.4699 - mse: 0.2656 - precision: 0.5517 - recall: 0.3200 - val_loss: 2.1468 - val_accuracy: 0.6667 - val_mse: 0.2384 - val_precision: 0.6250 - val_recall: 0.5556
Epoch 2/100
 1/11 [=>............................] - ETA: 0s - loss: 2.1711 - accuracy: 0.5000 - mse: 0.2508 - precision: 0.2500 - recall: 0.1667

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160036-qgxjbijr\files\model-best\assets


...

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160036-qgxjbijr\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160036-qgxjbijr\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160036-qgxjbijr\files\model-best)... Done. 0.0s


11/11 [==============================] - 3s 325ms/step - loss: 0.4040 - accuracy: 1.0000 - mse: 0.0064 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8505 - val_accuracy: 0.7619 - val_mse: 0.1657 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 58/100
11/11 [==============================] - 0s 8ms/step - loss: 0.3990 - accuracy: 1.0000 - mse: 0.0062 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8529 - val_accuracy: 0.7619 - val_mse: 0.1676 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 59/100
11/11 [==============================] - 0s 9ms/step - loss: 0.3935 - accuracy: 1.0000 - mse: 0.0062 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8546 - val_accuracy: 0.7619 - val_mse: 0.1699 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 60/100
11/11 [==============================] - 0s 12ms/step - loss: 0.3888 - accuracy: 1.0000 - mse: 0.0063 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8585 - val_accuracy: 0.7619 - val_mse: 0.1715 - val_precision: 0.6667 - val_r

INFO:tensorflow:Assets written to: ./model4\assets


INFO:tensorflow:Assets written to: ./model4\assets


In [20]:
w.finish()
w = wandb.init(project='zadanie1',reinit=True)

accuracy,▁▃▅▅▅▆▆▆▇▇▇█████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▇▆▆▅▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁▃▄▄▄▅▅▆▇▇▇▇████████████████████████████
recall,▁▄▆▇▇▇▇▇████████████████████████████████
val_accuracy,▁▆▃▆▆▆████████▆▆▆███▆▆▆▆▆▆▆▆▃▃▃▃▃▃▃▃▃▃▃▃
val_loss,█▇▆▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▇▅▄▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▅▅▅
val_precision,▁▄▂▃▃▃████████▄▄▄▆▆▆▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂
val_recall,▁▆▆███▆▆▆▆▆▆▆▆▆▆▆███████████▆▆▆▆▆▆▆▆▆▆▆▆


In [21]:
fitModel(tf_model5, 'model5', [earlyStop,WandbCallback()])

Epoch 1/100
 1/11 [=>............................] - ETA: 26s - loss: 0.8695 - accuracy: 0.3125 - mse: 0.3345 - precision: 0.6364 - recall: 0.3500

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160345-rcz18v8f\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160345-rcz18v8f\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160345-rcz18v8f\files\model-best)... Done. 0.0s


11/11 [==============================] - 6s 367ms/step - loss: 0.7740 - accuracy: 0.4819 - mse: 0.2813 - precision: 0.6667 - recall: 0.1600 - val_loss: 0.6618 - val_accuracy: 0.5714 - val_mse: 0.2353 - val_precision: 0.5000 - val_recall: 0.1111
Epoch 2/100
 1/11 [=>............................] - ETA: 0s - loss: 0.9371 - accuracy: 0.3750 - mse: 0.3311 - precision: 0.0000e+00 - recall: 0.0000e+00

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160345-rcz18v8f\files\model-best\assets


...

INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160345-rcz18v8f\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160345-rcz18v8f\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Burka\Documents\School\Letny semester 22-23\Neuronky\ns-zadanie1\wandb\run-20230329_160345-rcz18v8f\files\model-best)... Done. 0.0s


11/11 [==============================] - 3s 253ms/step - loss: 0.4325 - accuracy: 0.7952 - mse: 0.1408 - precision: 0.7938 - recall: 0.8462 - val_loss: 0.4552 - val_accuracy: 0.8095 - val_mse: 0.1538 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 43/100
11/11 [==============================] - 0s 11ms/step - loss: 0.4970 - accuracy: 0.7651 - mse: 0.1662 - precision: 0.7364 - recall: 0.8901 - val_loss: 0.4567 - val_accuracy: 0.7619 - val_mse: 0.1553 - val_precision: 0.6667 - val_recall: 0.8889
Epoch 44/100
11/11 [==============================] - 0s 11ms/step - loss: 0.4489 - accuracy: 0.8193 - mse: 0.1397 - precision: 0.8020 - recall: 0.8901 - val_loss: 0.4590 - val_accuracy: 0.8095 - val_mse: 0.1568 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 45/100
11/11 [==============================] - 0s 11ms/step - loss: 0.3843 - accuracy: 0.8313 - mse: 0.1242 - precision: 0.8539 - recall: 0.8352 - val_loss: 0.4575 - val_accuracy: 0.8095 - val_mse: 0.1568 - val_precision: 0.7273 - val

INFO:tensorflow:Assets written to: ./model5\assets


INFO:tensorflow:Assets written to: ./model5\assets


In [22]:
w.finish()
w = wandb.init(project='zadanie1',reinit=True)

accuracy,▁▁▁▃▃▃▃▄▄▄▅▅▅▄▄▅▅▅▅▆▆▆▆▆▆▇▇▆▆▇▇▇▇▇▇█▇▆▇█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,███▆▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▄▄▄▃▃▃▃▃▃▂▂▃▂▁▂▂▂▁
mse,███▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▄▄▄▃▃▃▃▂▃▂▂▂▂▁▂▂▂▁
precision,▄▁▂▃▃▄▂▄▃▄▄▅▄▄▄▄▄▅▅▆▆▆▅▅▆▆▆▆▆▇▇▆▇▇██▇▆▇▇
recall,▁▁▂▃▅▅▅▆▆▇▇▇▇▆▆▇▇▇▆▆▇▇▇▇▇█▇▇▇▇▇█▇█▇█▇▇██
val_accuracy,▁▂▂▂▁▄▄▅▅▅▅▅▇▇▇▇▇▇▇▅▇▇▅███▇▇▇▇▇████▇▇▇▇▇
val_loss,█████▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_mse,████▇▇▇▇▇▇▆▆▆▅▅▅▄▄▃▃▃▃▃▂▂▂▁▁▁▂▁▁▁▁▁▂▂▂▂▂
val_precision,▁▆▄▃▁▄▄▅▅▅▅▅▆▆▆▆▆▆▆▅▆▆▅███▇▇▇▇▇████▇▇▇▇▇
val_recall,▁▂▃▅▅▇▇████████████▇██▇███▇▇▇▇▇████▇▇▇▇▇


In [23]:
#pomocna funkcia
def predictEval(tf_model, XX, yy):
    # vykonanie predikcie
    y_pred = tf_model.predict(XX)
    # uprava outputu na boolean
    y_pred_bool = np.copy(y_pred)
    for x in y_pred_bool:
        x[0] = round(x[0])
    y_pred_bool

    #vratenie y a accuaracy
    return [y_pred, y_pred_bool, accuracy_score(y_pred_bool, yy), mean_squared_error(y_pred_bool, yy), precision_score(y_pred_bool, yy), recall_score(y_pred_bool, yy)]

Eval

In [24]:
def predictEvalWrap(model, name):
    train = predictEval(model, X_train, y_train)
    val = predictEval(model, X_val, y_val)
    test = predictEval(model, X_test, y_test)

    print(name)
    print('Accuracy score')
    print(f'Train: {train[2]*100:.2f}%')
    print(f'Val: {val[2]*100:.2f}%')
    print(f'Test: {test[2]*100:.2f}%')
    print('Mean squared error')
    print(f'Train: {train[3]*100:.2f}%')
    print(f'Val: {val[3]*100:.2f}%')
    print(f'Test: {test[3]*100:.2f}%')
    print('Precision')
    print(f'Train: {train[4]*100:.2f}%')
    print(f'Val: {val[4]*100:.2f}%')
    print(f'Test: {test[4]*100:.2f}%')
    print('Recall')
    print(f'Train: {train[5]*100:.2f}%')
    print(f'Val: {val[5]*100:.2f}%')
    print(f'Test: {test[5]*100:.2f}%')
    print('------------------------------------------')

In [25]:
predictEvalWrap(tf_model1, 'No overfit prevention')
predictEvalWrap(tf_model2, 'Dropout')
predictEvalWrap(tf_model3, 'Early stopping')
predictEvalWrap(tf_model4, 'Regularization')
predictEvalWrap(tf_model5, 'Dropout + Early stopping')

1/1 [==============================] - 0s 41ms/step
No overfit prevention
Accuracy score
Train: 100.00%
Val: 90.48%
Test: 76.19%
Mean squared error
Train: 0.00%
Val: 9.52%
Test: 23.81%
Precision
Train: 100.00%
Val: 100.00%
Test: 81.82%
Recall
Train: 100.00%
Val: 81.82%
Test: 75.00%
------------------------------------------
1/1 [==============================] - 0s 34ms/step
Dropout
Accuracy score
Train: 98.19%
Val: 85.71%
Test: 76.19%
Mean squared error
Train: 1.81%
Val: 14.29%
Test: 23.81%
Precision
Train: 96.70%
Val: 88.89%
Test: 81.82%
Recall
Train: 100.00%
Val: 80.00%
Test: 75.00%
------------------------------------------
1/1 [==============================] - 0s 48ms/step
Early stopping
Accuracy score
Train: 98.80%
Val: 71.43%
Test: 80.95%
Mean squared error
Train: 1.20%
Val: 28.57%
Test: 19.05%
Precision
Train: 97.80%
Val: 77.78%
Test: 90.91%
Recall
Train: 100.00%
Val: 63.64%
Test: 76.92%
------------------------------------------
1/1 [==============================] - 0s 48ms/

V rámci pytorch časti vytvárame 3 rôzne modely

Všetky modely majú rovnakú architektúru, ktorá sa líši iba v metódach použitých pri riešení problému overfittingu. Vstupná vrstva má 16 neurónov, nasledovaná dvoma skrytými vrstvami s 16 neurónmi a nakoniec výstupná vrstva s jedným neurónom.

Všetky vrstvy používajú aktivačnú funkciu ReLu okrem poslednej, kde sa používa sigmoid.

Model 1 nepoužíva žiadnu metódu prevencie overfittingu.

Model 2 používa metódu dropout, kde sme ako pravdepodobnosť vypadnutia neurónov zvolili 50%. Dropout sa aplikuje na prvú a druhú skrytú vrstvu.

Model 3 používa metódu early stopping, ktorá sa riadi podľa val_loss a parameter patience je nastavený na 10 epoch.

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

PyTorchModel nepoužíva žiadnu metódu prevencie overfittingu.

In [71]:
# Define the PyTorch model
class PyTorchModel(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

PyTorchModelDropout používa metódu dropout, kde sme ako pravdepodobnosť vypadnutia neurónov zvolili 50%. Dropout sa aplikuje na prvú a druhú skrytú vrstvu.

In [72]:
class PyTorchModelDropout(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.5):
        super(PyTorchModelDropout, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(16, 16)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(16, 16)
        self.dropout3 = nn.Dropout(dropout_rate)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout3(x)
        x = torch.sigmoid(self.fc4(x))
        return x

Táto trieda datasetu prijíma dva argumenty, X a y, ktoré sú príznaky a lable datasetu.
Metóda init() inicializuje dataset tým, že konvertuje príznaky a značky na PyTorch tenzory pomocou funkcie torch.tensor().

In [73]:
class SonarDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

Táto časť vytvára PyTorch datasety a dataloadery pre trénovacie, validačné a testovacie dáta pomocou vlastnej triedy datasetu "SonarDataset".

In [74]:
train_dataset = SonarDataset(X_train, y_train)
val_dataset = SonarDataset(X_val, y_val)
test_dataset = SonarDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

V tejto časti inicializujeme vyššie spomínané modely.

In [75]:
input_dim = len(X_train.columns)
model = PyTorchModel(input_dim)
model_dropout = PyTorchModelDropout(input_dim, dropout_rate=0.5)
model_early_stopping = PyTorchModel(input_dim)

Ako Loss funkcia sa používa trieda nn.BCELoss() a bude použitá ako stratová funkcia pre všetky tri modely. 
Ako optimizer sa používa Adam ktorý budú použitý na aktualizáciu váh a sklonov v rámci trénovania modelov.

In [76]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
optimizer_dropout = optim.Adam(model_dropout.parameters())
optimizer_early_stopping = optim.Adam(model_early_stopping.parameters())

In [77]:
num_epochs = 100

Táto časť trénuje model neurónovej siete pomocou trénovacích dát a vyhodnocuje výkon modelu na validačných dátach v každej epoch.

Trenóvanie modelu bez overfittingu.

In [78]:

for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        wandb.log({"loss": loss})
    with torch.no_grad():
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)

    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

Epoch: 1/100, Loss: 0.6834, Val Loss: 0.6948
Epoch: 2/100, Loss: 0.6822, Val Loss: 0.6937
Epoch: 3/100, Loss: 0.6801, Val Loss: 0.6909
Epoch: 4/100, Loss: 0.5474, Val Loss: 0.6859
Epoch: 5/100, Loss: 0.5811, Val Loss: 0.6811
Epoch: 6/100, Loss: 0.4940, Val Loss: 0.6736
Epoch: 7/100, Loss: 0.5534, Val Loss: 0.6570
Epoch: 8/100, Loss: 0.6035, Val Loss: 0.6379
Epoch: 9/100, Loss: 0.4733, Val Loss: 0.6123
Epoch: 10/100, Loss: 0.3010, Val Loss: 0.5725
Epoch: 11/100, Loss: 0.3578, Val Loss: 0.5332
Epoch: 12/100, Loss: 0.3875, Val Loss: 0.4771
Epoch: 13/100, Loss: 0.3024, Val Loss: 0.4405
Epoch: 14/100, Loss: 0.1250, Val Loss: 0.4095
Epoch: 15/100, Loss: 0.0572, Val Loss: 0.3799
Epoch: 16/100, Loss: 0.3194, Val Loss: 0.3705
Epoch: 17/100, Loss: 0.1074, Val Loss: 0.3654
Epoch: 18/100, Loss: 0.1490, Val Loss: 0.3592
Epoch: 19/100, Loss: 0.3101, Val Loss: 0.3667
Epoch: 20/100, Loss: 0.2206, Val Loss: 0.3717
Epoch: 21/100, Loss: 0.0173, Val Loss: 0.3620
Epoch: 22/100, Loss: 0.0899, Val Loss: 0.35

Trénovanie modelu s pridaným droupoutom.

In [88]:
for epoch in range(num_epochs):
    model_dropout.train()
    for X_batch, y_batch in train_loader:
        optimizer_dropout.zero_grad()
        y_pred = model_dropout(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        model_dropout.eval()
        val_loss_sum = 0
        val_batch_count = 0
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model_dropout(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)
            val_loss_sum += val_loss.item()
            val_batch_count += 1
        val_loss_avg = val_loss_sum / val_batch_count
    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss_avg:.4f}")

Epoch: 1/100, Loss: 0.7324, Val Loss: 0.6956
Epoch: 2/100, Loss: 0.7132, Val Loss: 0.6956
Epoch: 3/100, Loss: 0.6888, Val Loss: 0.6956
Epoch: 4/100, Loss: 0.6867, Val Loss: 0.6956
Epoch: 5/100, Loss: 0.6446, Val Loss: 0.6956
Epoch: 6/100, Loss: 0.7879, Val Loss: 0.6956
Epoch: 7/100, Loss: 0.6830, Val Loss: 0.6956
Epoch: 8/100, Loss: 0.7411, Val Loss: 0.6956
Epoch: 9/100, Loss: 0.6908, Val Loss: 0.6956
Epoch: 10/100, Loss: 0.6941, Val Loss: 0.6956
Epoch: 11/100, Loss: 0.7145, Val Loss: 0.6956
Epoch: 12/100, Loss: 0.7031, Val Loss: 0.6956
Epoch: 13/100, Loss: 0.6767, Val Loss: 0.6956
Epoch: 14/100, Loss: 0.6602, Val Loss: 0.6956
Epoch: 15/100, Loss: 0.7198, Val Loss: 0.6956
Epoch: 16/100, Loss: 0.7024, Val Loss: 0.6956
Epoch: 17/100, Loss: 0.6806, Val Loss: 0.6956
Epoch: 18/100, Loss: 0.6967, Val Loss: 0.6956
Epoch: 19/100, Loss: 0.7129, Val Loss: 0.6956
Epoch: 20/100, Loss: 0.7106, Val Loss: 0.6956
Epoch: 21/100, Loss: 0.7287, Val Loss: 0.6956
Epoch: 22/100, Loss: 0.7114, Val Loss: 0.69

Trénovanie modelu s early stoppage.

In [80]:
patience = 10

best_val_loss = float("inf")
counter = 0

for epoch in range(num_epochs):
    model_early_stopping.train()
    for X_batch, y_batch in train_loader:
        optimizer_early_stopping.zero_grad()
        y_pred = model_early_stopping(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        model_early_stopping.eval()
        val_loss_sum = 0
        val_batch_count = 0
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model_early_stopping(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)
            val_loss_sum += val_loss.item()
            val_batch_count += 1
        val_loss_avg = val_loss_sum / val_batch_count

    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss_avg:.4f}")

    # Early stopping logic
    if val_loss_avg < best_val_loss:
        best_val_loss = val_loss_avg
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping!")
            break

Epoch: 1/100, Loss: 0.6686, Val Loss: 0.7099
Epoch: 2/100, Loss: 0.6714, Val Loss: 0.7099
Epoch: 3/100, Loss: 0.6435, Val Loss: 0.7099
Epoch: 4/100, Loss: 0.7173, Val Loss: 0.7099
Epoch: 5/100, Loss: 0.6946, Val Loss: 0.7099
Epoch: 6/100, Loss: 0.7027, Val Loss: 0.7099
Epoch: 7/100, Loss: 0.7012, Val Loss: 0.7099
Epoch: 8/100, Loss: 0.6726, Val Loss: 0.7099
Epoch: 9/100, Loss: 0.6474, Val Loss: 0.7099
Epoch: 10/100, Loss: 0.7062, Val Loss: 0.7099
Epoch: 11/100, Loss: 0.6657, Val Loss: 0.7099
Early stopping!


Pomocná funkcia na vyhodnotenie úspešnosti.

In [89]:
def predict_eval(model, loader):
    with torch.no_grad():
        model.eval()
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            y_pred_bool = torch.round(y_pred)
            accuracy = accuracy_score(y_pred_bool, y_batch)
    return accuracy

In [90]:
train_accuracy = predict_eval(model, train_loader)
val_accuracy = predict_eval(model, val_loader)
test_accuracy = predict_eval(model, test_loader)

In [91]:
train_accuracy_dropout = predict_eval(model_dropout, train_loader)
val_accuracy_dropout = predict_eval(model_dropout, val_loader)
test_accuracy_dropout = predict_eval(model_dropout, test_loader)

In [92]:
train_accuracy_early_stopping = predict_eval(model_early_stopping, train_loader)
val_accuracy_early_stopping = predict_eval(model_early_stopping, val_loader)
test_accuracy_early_stopping = predict_eval(model_early_stopping, test_loader)

In [93]:
print('No overfit prevention')
print(f'Train: {train_accuracy * 100:.2f}%')
print(f'Val: {val_accuracy * 100:.2f}%')
print(f'Test: {test_accuracy * 100:.2f}%')

No overfit prevention
Train: 100.00%
Val: 80.95%
Test: 85.71%


In [94]:
print('Droupout')
print(f'Train: {train_accuracy_dropout * 100:.2f}%')
print(f'Val: {val_accuracy_dropout * 100:.2f}%')
print(f'Test: {test_accuracy_dropout * 100:.2f}%')

Droupout
Train: 16.67%
Val: 42.86%
Test: 42.86%


In [95]:
print('Early stopping')
print(f'Train: {train_accuracy_early_stopping * 100:.2f}%')
print(f'Val: {val_accuracy_early_stopping * 100:.2f}%')
print(f'Test: {test_accuracy_early_stopping * 100:.2f}%')

Early stopping
Train: 83.33%
Val: 42.86%
Test: 52.38%
